This note has snippets to use replay buffers of Tf-agents.
* See [tutorials](https://github.com/tensorflow/agents/blob/master/docs/tutorials/5_replay_buffers_tutorial.ipynb)

In [ ]:
import tensorflow as tf
from tf_agents.replay_buffers import tf_uniform_replay_buffer

In [ ]:
def createInstanceOfReplayBuffer(nPv = 3, nMv = 2, batch_size = 1, max_length=2**10):
    data_spec =  (
        tf.TensorSpec([nPv,], tf.float32, 'observation')
        , tf.TensorSpec([nMv,], tf.float32, 'action')
        , tf.TensorSpec([nPv,], tf.float32, 'next_observation')
        , tf.TensorSpec([], tf.float32, 'reward')
        )

    replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
        data_spec,
        batch_size=batch_size,
        max_length=max_length)
    
    return replay_buffer

In [ ]:
def collectData(nMv = 2, nPv = 3, batch_size = 1, nSample = 2**7):
    for _ in range(nSample):
        observationBatch = tf.random.normal([batch_size, nPv])
        actionBatch = tf.random.normal([batch_size, nMv])
        nextObservationBatch = tf.random.normal([batch_size, nPv])
        rewardBatch = tf.random.normal([batch_size,])
        yield (observationBatch, actionBatch, nextObservationBatch, rewardBatch)

create an instance of replay buffer:

In [ ]:
replay_buffer = createInstanceOfReplayBuffer()

add batches of items in the replay buffer:

In [ ]:
replay_buffer.clear()
for aBatch in collectData():
    replay_buffer.add_batch(aBatch)

read items from the buffer:

In [ ]:
sample_batch_size = 2**5
num_steps = 1
dataset = replay_buffer.as_dataset(
    sample_batch_size=sample_batch_size
    , num_steps=num_steps)
trajectories, _ = iter(dataset).__next__()

In [ ]:
# >>for trj in trajectories:
# >>    print(trj.shape)
#
# (32, 1, 3)
# (32, 1, 2)
# (32, 1, 3)
# (32, 1)